# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
print(db.establishments.find_one())

{'_id': ObjectId('6407717b180729d5d7a40477'), 'FHRSID': 647177, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000041489', 'BusinessName': 'Wear Bay Bowls Club', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': 'Wear Bay Road', 'AddressLine2': 'Folkestone', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT19 6PY', 'Phone': '', 'RatingValue': 4.0, 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2014-03-31T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10}, 'SchemeType': 'FHRS', 'geocode': {'longitude': 1.196408, 'latitude': 51.086058}, 'RightToReply': '', 'Distance': 4591.821311183521, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None,

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_data)

In [10]:
# Check that the new restaurant was inserted
result = establishments.find_one({"BusinessName":"Penang Flavours"})
print(result)

{'_id': ObjectId('6407797284f1bd239bd3c9ab'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': 0.08384, 'latitude': 51.490142}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True, 'longitude': None, 'latitude': None}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID':1, 'BusinessType':1}
result = establishments.find_one(query, fields)
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6407717b180729d5d7a4047c')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {"BusinessName":"Penang Flavours"},
    {'$set':
        {'BusinessTypeID': 1}
     }
)

In [13]:
# Confirm that the new restaurant was updated
query = {"BusinessName":"Penang Flavours"}
fields = {"BusinessName":1, 'BusinessTypeID':1, 'BusinessType':1}
results = establishments.find_one(query, fields)
pprint(results)

{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6407797284f1bd239bd3c9ab')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
query_dover = {'LocalAuthorityName': {'$regex': "Dover"}}
print("Number of documents in result:", establishments.count_documents(query_dover))

Number of documents in result: 0


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query_dover)

In [16]:
# Check if any remaining documents include Dover
query_dover = {'LocalAuthorityName': {'$regex': "Dover"}}
print("Number of documents in result:", establishments.count_documents(query_dover))

Number of documents in result: 0


In [17]:
# Check that other documents remain with 'find_one'
print(establishments.find_one())

{'_id': ObjectId('6407717b180729d5d7a40477'), 'FHRSID': 647177, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000041489', 'BusinessName': 'Wear Bay Bowls Club', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': 'Wear Bay Road', 'AddressLine2': 'Folkestone', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT19 6PY', 'Phone': '', 'RatingValue': 4.0, 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2014-03-31T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10}, 'SchemeType': 'FHRS', 'geocode': {'longitude': 1.196408, 'latitude': 51.086058}, 'RightToReply': '', 'Distance': 4591.821311183521, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None,

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude
establishments.update_many({}, [{"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}])

In [19]:
# Change the data type from String to Decimal for latitude
establishments.update_many({}, [{'$set': {'geocode.latitude': {'$toDouble': '$geocode.latitude'}}}])

In [20]:
# Check that the coordinates are now numbers
query = {}
fields = {'geocode.latitude': 1, 'geocode.longitude': 1}
limit = 5
# Capture the results to a variable
results = establishments.find(query, fields).limit(limit)

# Pretty print the results
for result in results:
    pprint(result)

{'_id': ObjectId('6407717b180729d5d7a40477'),
 'geocode': {'latitude': 51.086058, 'longitude': 1.196408}}
{'_id': ObjectId('6407717b180729d5d7a40479'),
 'geocode': {'latitude': 51.08084, 'longitude': 1.188537}}
{'_id': ObjectId('6407717b180729d5d7a4047a'),
 'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311705}}
{'_id': ObjectId('6407717b180729d5d7a4047b'),
 'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311705}}
{'_id': ObjectId('6407717b180729d5d7a4047c'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}
